This notebook will process isotherms selected previously to obtain KPI values.

In [1]:
import json
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

import pygaps

%cd ..

C:\Users\pauli\git\separation-explorer


Load isotherms from JSON files.

In [3]:
iso_path = r"./data/isotherms"
paths = pygaps.util_get_file_paths(iso_path, '.json')
isotherms = [pygaps.isotherm_from_jsonf(path) for path in paths]
print("Selected isotherms:", len(isotherms))

Selected isotherms: 5755


Compute Henry constant through two methods: initial slope and virial fitting.

In [4]:
no_henry_slope = []
no_henry_virial = []

for index, iso in enumerate(isotherms):
    try:
        iso.henry_slope = pygaps.initial_henry_slope(iso)
    except Exception as e:
        iso.henry_slope = None
        no_henry_slope.append(iso)

    try:
        iso.henry_virial = pygaps.initial_henry_virial(
            iso, optimization_params=dict(add_point=True))
    except Exception as e:
        iso.henry_virial = None
        no_henry_virial.append(iso)

# Only use isotherms which have the henry constant (slope mehtod) calculated going forward
isotherms = [iso for iso in isotherms if iso.henry_slope]
print("Isotherms with Henry constant:", len(isotherms))

c:\users\pauli\git\pygaps\src\pygaps\modelling\virial.py:188: UserWarning: Removed points which are equal to 0.
  warnings.warn('Removed points which are equal to 0.')


Isotherms with Henry constant: 5755


Compute uptake values on pre-determined pressures. Do not extrapolate above maximum range. If any value below minimum range, use Henry constant to compute loading.

In [5]:
no_loading_possible = []
model_possible = []

for index, iso in enumerate(isotherms):

    iso.uptake = {}

    prange = np.arange(0.5, 20.5, 0.5)
    minp = min(iso.pressure(branch='ads'))
    maxp = max(iso.pressure(branch='ads'))
    model = [a for a in prange if a < minp]
    direct = [a for a in prange if minp < a < maxp]

    try:
        for p in direct:
            iso.uptake[p] = np.asscalar(iso.loading_at(p))
    except Exception:
        no_loading_possible.append(iso)
        continue
    # Take the data between 0 and x from the henry model
    if model:
        for p in model:
            iso.uptake[p] = iso.henry_slope * p
        model_possible.append(iso)

    iso.uptake[0] = 0.  # we automatically include 0

Construct dictionary with results on a material basis.

In [7]:
materials = {}

gases = [
    'hydrogen', 'neon', 'argon', 'krypton', 'xenon',
    'methane', 'ethane', 'ethene', 'acetylene',
    'propane', 'propene',
    'butane', 'isobutane',
    '1-butene', 'cis-2-butene', 'trans-2-butene',
    'isobutene',
    'isopentane',
    'carbon dioxide', 'sulphur dioxide', 'nitrogen dioxide',
    'oxygen', 'carbon monoxide', 'nitrogen',
    'benzene', 'toluene',
    'water', 'methanol', 'ethanol', 'ammonia',
]

for mat in Counter([i.material for i in isotherms]).keys():
    materials[mat] = {
        gas: {
            'iso': [],
            'Kh': [],
            'L': []
        } for gas in gases}

for iso in isotherms:
    materials[iso.material][iso.adsorbate]['iso'].append(iso.filename)
    materials[iso.material][iso.adsorbate]['Kh'].append(iso.henry_slope)
    materials[iso.material][iso.adsorbate]['L'].append(iso.uptake)

Some outlier detection functions to be used later: gross outlier detection and interquartile outlier detection.

In [8]:
def kh_gross_outlier_rejection(a, l1=1e-7, l2=1e7):
    return [i for i in a if l1 < i < l2 and not np.isnan(i)]


def l_gross_outlier_rejection(a, l1=0, l2=1e7):
    return [i for i in a if l1 <= i < l2 and not np.isnan(i)]


def iqr_outlier_rejection(arr):
    q75, q25 = np.nanpercentile(sorted(arr), [75, 25], interpolation='linear')
    iqr = q75 - q25
    l_b = q25 - (1.5 * iqr)
    u_b = q75 + (1.5 * iqr)
    return [a for a in arr if a > l_b and a < u_b]

Outlier detection and median calculation for Henry constant.

In [9]:
for mat in materials:
    for gas in gases:
        K_hs = materials[mat][gas].get('Kh', None)
        if K_hs is None:
            materials[mat][gas]['mKh'] = np.nan
            materials[mat][gas]['eKh'] = np.nan
            materials[mat][gas]['lKh'] = np.nan

        K_hs = kh_gross_outlier_rejection(K_hs)

        if K_hs is None:
            materials[mat][gas]['mKh'] = np.nan
            materials[mat][gas]['eKh'] = np.nan
            materials[mat][gas]['lKh'] = np.nan

        nK_h = len(K_hs)
        if nK_h == 0:
            continue

        if nK_h > 4:
            K_hs = iqr_outlier_rejection(K_hs)
            mK_h = np.median(K_hs)
            eK_h = np.std(K_hs)
        elif nK_h == 1:
            mK_h = np.median(K_hs)
            eK_h = 0
        else:
            mK_h = np.median(K_hs)
            eK_h = np.std(K_hs)

        materials[mat][gas]['mKh'] = mK_h
        materials[mat][gas]['eKh'] = eK_h
        materials[mat][gas]['lKh'] = nK_h

Outlier detection and median calculation for uptake.

In [10]:
for mat in materials:
    for gas in gases:

        aL_s = materials[mat][gas].get('L', None)
        if not aL_s:
            materials[mat][gas]['mL'] = []
            materials[mat][gas]['eL'] = []
            materials[mat][gas]['lL'] = []
            continue

        pdL_s = pd.DataFrame(aL_s)

        materials[mat][gas]['mL'] = [0]
        materials[mat][gas]['eL'] = [0]
        materials[mat][gas]['lL'] = [0]

        for p in np.arange(0.5, 20.5, 0.5):
            try:
                L_s = l_gross_outlier_rejection(pdL_s[p])
                if L_s is None:
                    continue
            except Exception:
                continue

            nL_s = len(L_s)
            if nL_s == 0:
                materials[mat][gas]['mL'].append(np.nan)
                materials[mat][gas]['eL'].append(np.nan)
                materials[mat][gas]['lL'].append(np.nan)
                continue

            if nL_s > 4:
                L_s = iqr_outlier_rejection(L_s)
                mL_s = np.median(L_s)
                eL_s = np.std(L_s)
            elif nL_s == 1:
                mL_s = np.median(L_s)
                eL_s = 0
            else:
                mL_s = np.median(L_s)
                eL_s = np.std(L_s)

            materials[mat][gas]['mL'].append(mL_s)
            materials[mat][gas]['eL'].append(eL_s)
            materials[mat][gas]['lL'].append(nL_s)

Save the resulting json file.

In [11]:
save_path = r"./data/kpis.json"
with open(save_path, 'w') as file:
    json.dump(materials, file)